In [1]:
import warnings
warnings.filterwarnings('ignore')
import networkx as nx
import pandas as pd
from pprint import pprint

In [2]:
import numpy as np
from scipy.integrate import odeint
import matplotlib.pyplot as plt
from abc import ABC, abstractmethod # for later

def derive(y,t,N, beta, gamma):        
    """The SIR model differential equations."""

    S, I, R = y
    dSdt = -beta * S * I / N
    dIdt = beta * S * I / N - gamma * I 
    dRdt = gamma * I
    return dSdt, dIdt, dRdt
    
class SIR:
    
    def __init__(self, N=1000, I0=1, R0=0, beta=0.4, gamma=1./10, days=250):
        """Setup for the model.
        
        N:     Total population, N.
        I0,R0: Initial number of infected and recovered individuals.
        beta:  Contact rate, beta, 
        gamma: Mean recovery rate,(in 1/days).
        t:     A grid of time points (in days)
        """
        
        self.N  = N
        self.I0 = I0
        self.R0 = R0
        self.S = 0
        self.I = 0
        self.R = 0
        self.beta = beta
        self.gamma = gamma
        self.t = np.linspace(0, days, days) 
    
    def projection(self):
        """Expected change in the disease's distribution in the population. """
        
        S0 = self.N - self.I0 - self.R0 # Everyone else, S0, is susceptible to infection initially.
        # Initial conditions vector
        y0 = S0, self.I0, self.R0
        # Integrate the SIR equations over the time grid, t.
        ret = odeint(derive, y0, self.t, args=(self.N, self.beta, self.gamma))
        self.S, self.I, self.R = ret.T

    def plot_projection(self):
        """Plot the data on three separate curves for S(t), I(t) and R(t)"""
        
        fig = plt.figure(facecolor='w')
        ax = fig.add_subplot(111, facecolor='#dddddd', axisbelow=True)
        ax.plot(self.t, self.S/1000, 'b', alpha=0.5, lw=2, label='Susceptible')
        ax.plot(self.t, self.I/1000, 'r', alpha=0.5, lw=2, label='Infected')
        ax.plot(self.t, self.R/1000, 'g', alpha=0.5, lw=2, label='Recovered with immunity')
        ax.set_xlabel('Time /days')
        ax.set_ylabel('Number (1000s)')
        ax.set_ylim(0,1.2)
        ax.yaxis.set_tick_params(length=0)
        ax.xaxis.set_tick_params(length=0)
        ax.grid(b=True, which='major', c='w', lw=2, ls='-')
        legend = ax.legend()
        legend.get_frame().set_alpha(0.5)
        for spine in ('top', 'right', 'bottom', 'left'):
            ax.spines[spine].set_visible(False)
        plt.show()
        
class City(SIR):
    
    def __init__(self, name, position, area, air, port):
        super().__init__()
        self.name = name
        self.position = position
        self.area = area
        self.air = air
        self.port = port
    
    def get_name(self):
        return self.name
    
    def get_position(self):
        return self.position
    
    def __repr__(self):
         return "name: " + self.name + ",\t\tarea: " + str(self.area)

In [3]:
import networkx as nx
import pandas as pd
import random, json
import numpy as np, numpy.random
from unidecode import unidecode
#from City import City 
class CountryGraph:
    """
    Creates a distribution of the cities spatially based on data.
    
    To represent each city properly we need to pass them as nodes, 
    and put several factors that suggest the movement of the people
    in them.
    """
    
    
    def __init__(self, country="Turkey"):
        """
        country: the country's name, that will be read from a configuration file.
        subs:    the cities and their sub-cities which will be used for mapping back.
        G:       the graph containing the cities. 
        """
        
        self.country = country
        self.subs = {}
        self.G = nx.Graph()
        # setup
        self.__read_cities()
        # transportation
        self.__update_roads()
        #self.__update_ports()

    def __read_cities(self):
        """Update all the cities, and the sub-cities connected to them."""
        
        self.cities = pd.read_csv("geo_data/cities_" + self.country.lower() +".csv")
        cities_subs = pd.read_csv("geo_data/cities_" + self.country.lower() + "_subs.csv")
        for e in range(len(cities_subs)):
            self.subs[ cities_subs.iloc[e]['city']] = cities_subs.iloc[e]['admin']
        for city in list(self.cities['city']):
            city_properties = self.cities[self.cities['city'] == city].iloc[0]
            self.G.add_node(City( 
                name=city_properties['city'],
                position=(city_properties['lat'], city_properties[ 'lng']),
                area=int(city_properties['Area(km²)'].replace(',','')), 
                air=0, 
                port=0))
    
    def __update_roads(self):
        """Create the connections in the graph which represent roads."""
        
        with open("geo_data/roads_turkey") as f:
            self.roads = list(json.loads( json.load( f)).values())
        road_cities = []
        for road in self.roads:
            road = self.remove_duplicates([self.subs[i] for i in road])
            road_cities += road # add the cities which are connected to this road
            for i in range(len(road)-1):
                self.G.add_edge( road[i], road[i+1], weight=10)
        road_cities = set(road_cities)
        not_found = set( self.cities['city']) - set(road_cities)
        # order cities present in list by distance to this city
        for each in not_found:
            city = self.cities[ self.cities['city'] == each].iloc[0]
            found = self.cities[ self.cities['city'].isin( road_cities)]
            found['distance'] = ((found['lat']-city['lat'])**2 
                                 + (found['lng']-city['lng'])**2)
            closest_city = found.sort_values('distance').iloc[1]['city']
            print(road[i], road[i+1])
            self.G.add_edge( city['city'], closest_city, weight=10)
    
    def display_graph(self):
        pos = {city:(long, lat) 
               for (city, (lat,long)) in nx.get_node_attributes(self.G, 'position').items()}
        nx.draw(self.G, pos, with_labels=False, 
                node_size=10, node_color='r', edge_color='b')
       
    def __update_ports(self):
        with open("geo_data/ports_turkey.json") as f:
            ports = json.load(f)
        for each in ports['airport']:
            self.G.nodes[self.subs[each]]['air'] = 1
        for each in ports['port']:
            self.G.nodes[self.subs[each]]['port'] = 1
    
    @staticmethod
    def remove_duplicates(seq):
        seen = set()
        seen_add = seen.add
        return [x for x in seq if not (x in seen or seen_add(x))]

In [4]:
gTurkey = CountryGraph()
pprint([each for each in gTurkey.G.nodes])

'Edirne'
İstanbul
WTF
Kocaeli
WTF
Sakarya
WTF
Düzce
WTF
Bolu
WTF
Çankırı
WTF
Amasya
WTF
Tokat
WTF
Erzincan
WTF
Erzurum
WTF
'Çanakkale'
Balıkesir
WTF
Bursa
WTF
Eskişehir
WTF
Ankara
WTF
Kırıkkale
WTF
Yozgat
WTF
Sivas
WTF
'İzmir'
Manisa
WTF
Uşak
WTF
Afyonkarahisar
WTF
Konya
WTF
Aksaray
WTF
Nevşehir
WTF
Kayseri
WTF
Sivas
WTF
Malatya
WTF
Elazığ
WTF
Bingöl
WTF
Muş
WTF
Bitlis
WTF
'Muğla'
Antalya
WTF
Mersin
WTF
Adana
WTF
Gaziantep
WTF
Şanlıurfa
WTF
Şırnak
WTF
Sakarya
WTF
Düzce
WTF
Zonguldak
WTF
Bartın
WTF
Kastamonu
WTF
Sinop
WTF
Samsun
WTF
Ordu
WTF
Giresun
WTF
Trabzon
WTF
Rize
WTF
Artvin
WTF
Ardahan
WTF
Kars
WTF
Edirne
WTF
Çanakkale
WTF
Van
WTF
İzmir
WTF
Aydın
WTF
Sakarya
WTF
Bilecik
WTF
Kütahya
WTF
Afyonkarahisar
WTF
Burdur
WTF
Zonguldak
WTF
Bolu
WTF
Ankara
WTF
Aksaray
WTF
Adana
WTF
Ordu
WTF
Tokat
WTF
Sivas
WTF
Malatya
WTF
Gaziantep
WTF
Artvin
WTF
Erzurum
WTF
Bingöl
WTF
Diyarbakır
WTF
Diyarbakır Mardin
Diyarbakır Mardin
Diyarbakır Mardin
Diyarbakır Mardin
Diyarbakır Mardin
Diyarbakır Mardin
D

In [5]:
#gTurkey.display_graph()

In [6]:
[each for each in gTurkey.G.edges]

[('Edirne', 'İstanbul'),
 ('Edirne', 'Çanakkale'),
 ('İstanbul', 'Kocaeli'),
 ('Kocaeli', 'Sakarya'),
 ('Sakarya', 'Düzce'),
 ('Sakarya', 'Bilecik'),
 ('Düzce', 'Bolu'),
 ('Düzce', 'Zonguldak'),
 ('Bolu', 'Çankırı'),
 ('Bolu', 'Zonguldak'),
 ('Bolu', 'Ankara'),
 ('Çankırı', 'Amasya'),
 ('Amasya', 'Tokat'),
 ('Amasya', 'Çorum'),
 ('Tokat', 'Erzincan'),
 ('Tokat', 'Ordu'),
 ('Tokat', 'Sivas'),
 ('Erzincan', 'Erzurum'),
 ('Erzincan', 'Sivas'),
 ('Erzincan', 'Bayburt'),
 ('Erzincan', 'Gümüşhane'),
 ('Erzincan', 'Tunceli'),
 ('Erzurum', 'Ağrı'),
 ('Erzurum', 'Artvin'),
 ('Erzurum', 'Bingöl'),
 ('Çanakkale', 'Balıkesir'),
 ('Çanakkale', 'Van'),
 ('Çanakkale', 'Kırklareli'),
 ('Çanakkale', 'Tekirdağ'),
 ('Balıkesir', 'Bursa'),
 ('Bursa', 'Eskişehir'),
 ('Bursa', 'Yalova'),
 ('Eskişehir', 'Ankara'),
 ('Ankara', 'Kırıkkale'),
 ('Ankara', 'Aksaray'),
 ('Kırıkkale', 'Yozgat'),
 ('Yozgat', 'Sivas'),
 ('Sivas', 'Kayseri'),
 ('Sivas', 'Malatya'),
 ('İzmir', 'Manisa'),
 ('İzmir', 'Van'),
 ('İzmir', '